<a href="https://colab.research.google.com/github/cpyp-ipdp/pgd_cdmx/blob/main/indicadores/educacion_ciencia/indicadores_educ_ciencia_tec_pgd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
##Estimación de indicadores de vivienda
#Se borra todo lo que se encuentra en el entorno

rm(list=ls())

# Librerías ====
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, showtext, srvyr,googledrive,haven)

Loading required package: pacman



In [14]:
#Crear directorio para guardar los archivos
dir.create("data", showWarnings = FALSE)

In [ ]:
url_basica<-"https://www.inegi.org.mx/contenidos/programas/enigh/nc/"
segunda_parte<-"microdatos/enigh"
fin1<-"_ns_"
fin2<-"_csv.zip"

In [ ]:
years<-c("2016", "2018", "2020", "2022")

In [ ]:
# Función genérica para descargar, descomprimir, renombrar y eliminar
procesar_archivo <- function(year, tipo, url_base, segunda_parte, fin1, fin2, carpeta_destino) {
  # Construir URL y ruta de destino
  url <- paste0(url_base, year, "/", segunda_parte, year, fin1, tipo, fin2)
  destfile <- paste0(carpeta_destino, "/", year, fin1, tipo, fin2)

  # Descargar el archivo
  download.file(url, destfile = destfile)

  # Descomprimir el archivo
  unzip(destfile, exdir = carpeta_destino)

  # Renombrar el archivo descomprimido
  file.rename(
    from = paste0(carpeta_destino, "/", tipo, ".csv"),
    to = paste0(carpeta_destino, "/", tipo, "_", year, ".csv")
  )

  # Eliminar el archivo descargado
  file.remove(destfile)
}

In [ ]:
# Iterar sobre los años y procesar cada tipo de archivo
for (i in 1:length(years)) {
  procesar_archivo(years[i], "poblacion", url_basica, segunda_parte, fin1, fin2, "data")
  procesar_archivo(years[i], "viviendas", url_basica, segunda_parte, fin1, fin2, "data")
}

#### Función de estimación

In [ ]:
estimacion <- function(path, variable) {
      # Cargar los datos de población
  datos <- read_csv(path, show_col_types = FALSE)

  # Si son datos de 2016 a 2020, unir con la tabla de viviendas
  if (substr(path, 16, 19) %in% c("2016", "2018", "2020")) {
    # Cargar tabla viviendas
    vivienda <- read_csv(str_replace(path, "poblacion", "viviendas"), show_col_types = FALSE)

    # Pegar factor, upm y est_dis de la tabla vivienda
    datos <- datos %>%
      left_join(vivienda %>% select(folioviv, factor, upm, est_dis), by = "folioviv")
  }

  # Cargar los datos
  datos <- datos %>%
    mutate(cve_ent = case_when(
      nchar(folioviv) == 9  ~ paste0("0", substr(folioviv, 1, 1)),
      nchar(folioviv) == 10 ~ substr(folioviv, 1, 2)
    )) %>%
    # Filtrar si entidad es 9
  filter(cve_ent == "09")


  # Definir diseño muestral
  mydesign <- datos %>%
    as_survey_design(ids = upm,
                     strata = est_dis,
                     weights = factor)


###Estimaciones###
  # Población de 6 a 14 años que asiste a la escuela
  asistencia <- mydesign %>%
filter(edad >= 6 & edad <= 14)%>%
    group_by(asis_esc) %>%
    summarise(
      personas = survey_total(vartype = "cv"),
      porcentaje = survey_prop(vartype = "cv")
    ) %>%
    mutate(porcentaje = round(porcentaje * 100, 2)) %>%
    filter(asis_esc == 1) %>%
    # Añadir año
    mutate(year = substr(path, 16, 19))


#Población mayor a 3 años analfabeta
    analfabetismo <- mydesign %>%
filter(edad>3)%>%
        group_by(alfabetism) %>%
        summarise(
        personas = survey_total(vartype = "cv"),
        porcentaje = survey_prop(vartype = "cv")
        ) %>%
        mutate(porcentaje = round(porcentaje * 100, 2)) %>%
        filter(alfabetism == 2) %>%
        # Añadir año
        mutate(year = substr(path, 16, 19))



##Grado promedio de escolaridad
        gradoprom<-mydesign%>%
filter(!is.na(nivelaprob))%>%
    filter(nivelaprob>=2)%>%
#Filtrar 15 años y más
    filter(edad>=15)%>%
#Calcular años de escolaridad según nivel y grados aprobados
    mutate(
        anos_escolaridad=
        case_when(
            #Primaria
            nivelaprob==2 ~ gradoaprob,
            #Secundaria
            nivelaprob==3 ~ gradoaprob+6,
            #Preparatoria
            nivelaprob==4 ~ gradoaprob+9,
#Licenciatura, normal o carrera técnica
            nivelaprob %in% c(5,6,7) ~ gradoaprob+12,
#Posgrado (nivelaprob 8,9)
            nivelaprob %in% c(8,9) ~ gradoaprob+17
        )


    )%>%
    summarise(
        promedio_escolaridad=survey_mean(anos_escolaridad,na.rm = TRUE, vartype = "cv")
    )%>%
    mutate(year=substr(path,16,19))






  # Devolver el resultado basado en la variable
  if (variable == "asistencia_escolar") {
    return(asistencia)
  }
    else if (variable == "analfabetismo") {
    return(analfabetismo)
    }
    else if (variable == "grado_escolaridad") {
    return(gradoprom)
    }
else {
    stop("Variable no reconocida. Use 'rezago' o 'sin_escrituras'.")
  }

}


### Indicadores educativos
#### Porcentaje de población de 6 a 14 años que asiste a la escuela

In [ ]:
tabla_asistencia<-map_dfr(
  list.files("data", full.names = TRUE,pattern = "poblacion_.*.csv"),
  estimacion,
  variable = "asistencia_escolar"
)
print("Porcentaje de población de 6 a 14 años que asiste a la escuela")
tabla_asistencia

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"


asis_esc,personas,personas_cv,porcentaje,porcentaje_cv,year
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,1174501,0.04769559,99.11,0.003358606,2016
1,1134481,0.04667804,98.98,0.005532614,2018
1,1022955,0.04437516,97.91,0.005801212,2020
1,1050982,0.04475938,98.79,0.003823138,2022


#### Porcentaje de población mayor a 3 años analfabeta

In [ ]:
tabla_analfabetismo<-map_dfr(
  list.files("data", full.names = TRUE,pattern = "poblacion_.*.csv"),
  estimacion,
  variable = "analfabetismo"
)
print("Porcentaje de población mayor a 3 años analfabeta")
tabla_analfabetismo

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"


[1] "Porcentaje de población mayor a 3 años analfabeta"


alfabetism,personas,personas_cv,porcentaje,porcentaje_cv,year
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
2,392513,0.07218314,4.48,0.06826306,2016
2,385054,0.06821630,4.33,0.06324768,2018
2,362758,0.06924958,4.05,0.06339509,2020
2,318218,0.07606005,3.50,0.07296126,2022



#### Grado promedio de escolaridad

In [ ]:
tabla_gradoprom<-map_dfr(
  list.files("data", full.names = TRUE,pattern = "poblacion_.*.csv"),
  estimacion,
  variable = "grado_escolaridad"
)
print("Grado promedio de escolaridad de la población de 15 años y más")
tabla_gradoprom

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"


[1] "Grado promedio de escolaridad de la población de 15 años y más"


promedio_escolaridad,promedio_escolaridad_cv,year
<dbl>,<dbl>,<chr>
11.56641,0.007964465,2016
11.67076,0.007571768,2018
11.76720,0.006940187,2020
11.95221,0.006589243,2022


#### Porcentaje de población en pobreza por rezago educativo

In [15]:
  #Conectar a google drive
drive_auth(email = "claudiodanielpc@gmail.com",cache = FALSE)

Please point your browser to the following url: 

https://accounts.google.com/o/oauth2/v2/auth?client_id=603366585132-frjlouoa3s2ono25d2l9ukvhlsrlnr7k.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email&redirect_uri=https%3A%2F%2Fwww.tidyverse.org%2Fgoogle-callback%2F&response_type=code&state=aeac4526bbef34c54aefa9b71442f263&login_hint=claudiodanielpc%40gmail.com&access_type=offline&prompt=consent



Enter authorization code: eyJjb2RlIjoiNC8wQVNWZ2kzS2wzZUxmQUpsLVRpNWhNazd3YWV6bkZLaHR1Tm5nQ1JsTy1yUUpONW02cEtJRFVkQU5iVkpQMmhJN0RnT3ZrZyIsInN0YXRlIjoiYWVhYzQ1MjZiYmVmMzRjNTRhZWZhOWI3MTQ0MmYyNjMifQ==


In [16]:
carpeta<-as_id(drive_get("https://drive.google.com/drive/folders/1TBfoik4w4DWhwFqWeZFgkZvrBdDbqzku"))


archivos<-drive_ls(carpeta)
print(archivos)

# A dribble: 3 × 3
  name             id                                drive_resource   
  <chr>            <drv_id>                          <list>           
1 pobreza_2022.dta 1T_WfYanYMOoWWbhj_CeiqxYRNrpXmDan <named list [42]>
2 pobreza_2020.dta 1TXJLNA_itWfjsGIdoj1bJQ4erNjc1wIx <named list [42]>
3 pobreza_2018.dta 1TTRfiRDGWgQwHylvduHx1IRdm5vQ3VeL <named list [42]>


In [19]:
estimacion_pobreza <- function(file_id, file_name) {
  # Descarga de datos
  local_path <- file.path("data", file_name)
  drive_download(as_id(file_id), path = local_path, overwrite = TRUE)

  # Leer datos
  datos <- read_dta(local_path)

  # Añadir año
  year <- str_extract(file_name, "\\d{4}")


  # Diseño muestral
  mydesign <- datos %>%
    as_survey_design(ids = upm,
                     strata = est_dis,
                     weights = factor)

  # Estima pobreza
  pobreza <- mydesign %>%
    #Filtrar datos para CDMX
  filter(entidad==9)%>%
group_by(pobreza_rei)%>%
    #Filtar grupo de población en pobreza

    summarise(
      poblacion = survey_total(vartype = "cv"),
      porcentaje = survey_prop(vartype = "cv")
    ) %>%
    mutate(porcentaje = round(porcentaje * 100, 2),
           year = as.numeric(year))%>%
filter(pobreza_rei == 1)

  return(pobreza)
}

In [20]:
tabla_pobreza <- map_dfr(
  seq_len(nrow(archivos)),
  ~ estimacion_pobreza(archivos$id[.x], archivos$name[.x])
)%>%
  arrange(year)

print("Porcentaje de población en pobreza por rezago educativo")
tabla_pobreza

File downloaded:

• pobreza_2022.dta <id: 1T_WfYanYMOoWWbhj_CeiqxYRNrpXmDan>

Saved locally as:

• data/pobreza_2022.dta

File downloaded:

• pobreza_2020.dta <id: 1TXJLNA_itWfjsGIdoj1bJQ4erNjc1wIx>

Saved locally as:

• data/pobreza_2020.dta

File downloaded:

• pobreza_2018.dta <id: 1TTRfiRDGWgQwHylvduHx1IRdm5vQ3VeL>

Saved locally as:

• data/pobreza_2018.dta



[1] "Porcentaje de población en pobreza por rezago educativo"


pobreza_rei,poblacion,poblacion_cv,porcentaje,porcentaje_cv,year
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2338591,0.03957730,25.43,0.03024622,2018
1,2233119,0.03768032,24.17,0.03081337,2020
1,2225816,0.03786385,23.92,0.03114584,2022
